In [0]:
from pyspark.sql.functions import explode, col
import requests, json

In [0]:
token = dbutils.secrets.get(scope="hubspot_secrets", key="api_token")

In [0]:
 
url = 'https://api.hubapi.com/crm/v3/objects/contacts'
headers = {'Authorization': F'Bearer {token}'}
data = requests.get(url, headers=headers).json()
dbutils.fs.put('abfss://landing@salesmarketingstorage1.dfs.core.windows.net/hubspot/contacts.json', json.dumps(data), overwrite=True)

In [0]:
df = (
spark.read 
.options(header=True)
.option("multiline", "true") 
.json("abfss://landing@salesmarketingstorage1.dfs.core.windows.net/hubspot/contacts.json")
)

df.show()

In [0]:
df.printSchema()

In [0]:


exploded_df = df.select(explode(col("results")).alias("contact"))
exploded_df.printSchema()

In [0]:
flat_df = exploded_df.select(
    col("contact.id").alias("contact_id"),
    col("contact.archived").alias("archived"),
    col("contact.createdAt").alias("created_at"),
    col("contact.updatedAt").alias("updated_at"),
    col("contact.url").alias("url"),
    col("contact.properties.firstname").alias("firstname"),
    col("contact.properties.lastname").alias("lastname"),
    col("contact.properties.email").alias("email"),
    col("contact.properties.createdate").alias("property_createdate"),
    col("contact.properties.lastmodifieddate").alias("property_lastmodifieddate"),
    col("contact.properties.hs_object_id").alias("hs_object_id")
)
flat_df.display()

In [0]:
flat_df.write.mode("overwrite").saveAsTable("bronze.hubspot.hubspot_contacts")